In [3]:
import pandas as pd
import numpy as np
from yahooquery import Ticker
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date, timedelta
import altair as alt

In [40]:
link = ("https://en.wikipedia.org/wiki/DAX#Components")
dax = pd.read_html(link)[4]
dax_new = [str(stock) for stock in dax["Ticker"]]

In [26]:
today = date.today()
start_date = date.today() - timedelta(weeks=27)
start_date = f"{start_date.year}-{start_date.month}-{start_date.day}"
today = f"{today.year}-{today.month}-{today.day}"
momentum_start = date.today() - timedelta(weeks=4)
momentum_start = f"{momentum_start.year}-{momentum_start.month}-{momentum_start.day}"
start_date, today, momentum_start, today

('2023-3-22', '2023-9-27', '2023-8-30', '2023-9-27')

In [27]:
def compass(stocks):
    _data_points = ["Stock","Momentum", "Levy"]
    updated_stocks = []
    for stock in stocks:
        if len(pd.DataFrame(yf.Ticker(stock).history(start=start_date))["Close"]) != 0:
            updated_stocks.append(stock)
        else:
            pass
    data = pd.DataFrame({"Stock": updated_stocks, "Momentum": None, "Levy": None}, index=updated_stocks)
    for stock in updated_stocks:
        data.loc[stock]["Stock"] = pd.DataFrame(dax).set_index(dax["Ticker"]).loc[stock]["Company"]
        data.loc[stock]["Momentum"] = momentum(stock)
        data.loc[stock]["Levy"] = relative_strength(stock)
    return data
      
def relative_strength(stock, start=start_date):
    constructor = pd.DataFrame(yf.Ticker(stock).history(start=start))
    strength = constructor["Close"][-1] / np.mean(constructor["Close"][:-1])
    return strength

def momentum(stock, start=momentum_start):
    constructor = pd.DataFrame(yf.Ticker(stock).history(start=start))
    moment = constructor["Close"][-1] / constructor["Close"][0]
    return moment

In [37]:
data = compass(dax_new)

In [45]:
#### Visualization Option 1
base = alt.Chart(data)
xscale = alt.Scale(domain=(0.5, 1.5))
yscale = alt.Scale(domain=(0.5, 1.5))

points = base.mark_circle(color="red").encode(
    alt.X("Levy:Q").scale(xscale),
    alt.Y("Momentum:Q").scale(yscale),
)

text = points.mark_text(align='left',baseline='middle',dx=10).encode(
    text="Stock",
)


#y_line = alt.Chart(pd.DataFrame({'x': [1]})).mark_rule().encode(y='x')
#x_line = alt.Chart(pd.DataFrame({"y":[1]})).mark_rule().encode(x="y")
x_line = alt.Chart().mark_rule(strokeDash=[10, 10]).encode(y=alt.datum(1))
y_line = alt.Chart().mark_rule(strokeDash=[10, 10]).encode(x=alt.datum(1))


(points+text+x_line+y_line).properties(width=650,height=500, title="Trending Stock Indicator").interactive()

alt.LayerChart(...)

In [42]:
#### Visualization Option 2
points = alt.Chart(data).mark_point(color="red").encode(
    alt.X("Levy:Q", scale=alt.Scale(domain=[0.5, 1.5])),
    alt.Y("Momentum:Q", scale=alt.Scale(domain=[0.5, 1.5]))
)

text = points.mark_text(align='left',baseline='middle',dx=10).encode(
    text="Stock",
)
#y_line = alt.Chart(pd.DataFrame({'x': [1]})).mark_rule().encode(y='x')
#x_line = alt.Chart(pd.DataFrame({"y":[1]})).mark_rule().encode(x="y")
x_line = alt.Chart().mark_rule(strokeDash=[10, 10]).encode(y=alt.datum(1))
y_line = alt.Chart().mark_rule(strokeDash=[10, 10]).encode(x=alt.datum(1))


(points+text+x_line+y_line).properties(width=650,height=500, title="Trending Stock Indicator").interactive()

alt.LayerChart(...)

In [43]:
data["Mean"] = np.mean([data["Momentum"], data["Levy"]], axis=0)
new_data = data.sort_values(by=["Mean"], axis=0, ascending=False)[:30]

In [44]:
main_chart = alt.Chart(new_data, title="List of Best 30 Stocks").mark_point(color="red").encode(
alt.X("Mean:Q")
    .title("Levy: blue, Mean: red, Momentum: green")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .title("")
    .sort("-x")
    .axis(grid=True),
#alt.Color("Mean:Q")
#    .legend(title="Mean"),
)

chart_1= alt.Chart(new_data).mark_point(color="green").encode(
alt.X("Momentum:Q")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .sort()
    .axis(grid=True),
#alt.Color("Momentum:Q")
#    .legend(title="Momentum")
)

chart_2 = alt.Chart(new_data).mark_point(color="blue").encode(
alt.X("Levy:Q")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .sort()
    .axis(grid=True),

#alt.Color("Levy")
#    .legend(title="Levy")
)
(main_chart+chart_1+chart_2).properties(height=alt.Step(20)).configure_view(stroke="transparent")#.save("bestperf.png", scale_factor=4)

alt.LayerChart(...)